In [1]:
import pandas as pd
import pathlib
import numpy as np
import h5py

In [2]:
target_dir=pathlib.Path("/media/mjsobrep/43CDA61E672B9161/pose/")
data_labels=pd.read_csv(target_dir/"data_labels.csv")
hdf5_file = h5py.File(target_dir/"smoothed_data.hdf5",'r')

In [3]:
data_labels=data_labels[data_labels['record_id'].isin([int(id) for id in hdf5_file if id[0]=='0'])]

In [4]:
data_labels['min_bbt']=data_labels.filter(items=['bbt.z_left','bbt.z_right']).min(axis=1)

For each age group, select 1 subject randomly below median and one above. Those are the test set

In [5]:
children=data_labels[data_labels['age']<12]
ya=data_labels[(12<=data_labels['age'])& (data_labels['age']<21)]
adult=data_labels[(21<=data_labels['age'])& (data_labels['age']<65)]
elder=data_labels[(65<=data_labels['age'])]

In [6]:
def rand_select(data,rng):
    med = data['min_bbt'].median()
    lower_group=data[data['min_bbt']<=med]
    lower_test=lower_group.sample(1, random_state=rng)
    lower_train=lower_group.drop(lower_test.index)
    upper_group=data[data['min_bbt']>med]
    upper_test=upper_group.sample(1, random_state=rng)
    upper_train=upper_group.drop(upper_test.index)
    test=pd.concat([lower_test,upper_test])
    train=pd.concat([lower_train, upper_train])
    return train,test
    
rng=np.random.RandomState(24402)
children_train, children_test = rand_select(children,rng)
ya_train, ya_test=rand_select(ya,rng)
adult_train, adult_test=rand_select(adult,rng)
elder_train, elder_test=rand_select(elder,rng)

train=pd.concat([children_train, ya_train, adult_train, elder_train])
test=pd.concat([children_test, ya_test, adult_test, elder_test])

In [7]:
train

,Unnamed: 0,record_id,age,bbt.z_left,bbt.z_right,min_bbt
9,10,23,4,-1.034091,-2.561644,-2.561644
25,26,44,7,-3.071429,-10.169811,-10.169811
3,4,15,5,0.396552,0.507463,0.396552
27,28,46,18,-3.764706,-3.712766,-3.764706
28,29,48,15,-9.443038,-5.241379,-9.443038
36,37,63,14,-5.139241,-4.781609,-5.139241
24,25,43,14,-3.367089,-2.942529,-3.367089
29,30,49,16,-2.670330,-2.333333,-2.670330
37,38,64,13,-2.365854,-2.481928,-2.481928
1,2,13,62,-2.654321,-4.806818,-4.806818


In [8]:
test

,Unnamed: 0,record_id,age,bbt.z_left,bbt.z_right,min_bbt
22,23,41,8,-9.408163,-6.837209,-9.408163
18,19,36,10,-2.191176,-2.231884,-2.231884
26,27,45,16,-9.921569,-9.229885,-9.921569
23,24,42,19,-1.647059,-2.436170,-2.436170
20,21,39,48,-3.197368,-3.826667,-3.826667
2,3,14,31,-1.395062,-0.322222,-1.395062
19,20,38,67,-2.636364,-3.064516,-3.064516
17,18,35,70,-3.042857,-1.085714,-3.042857


In [9]:
test.to_csv(target_dir/'test.csv')
train.to_csv(target_dir/'train.csv')